# Synopsis

PURPOSE :

Aller récupérer sur l'API de PVOutput.org les données d'une station cible:
- données production "live" (cad pas de temps de 5min à 10min sans la météo) ==> Detail_PROD
- données d'insolation (cad pas de temps de 5min à 10min sans la météo. Pas complètement clair si correspond exactement extraterestrial radiation) ==> Detail_INSOL
- données production quotidienne (somme sur la journée, dispo aussi en mensuel etc. Ajoute un champ (catégoriel) sur la couverture nuageuse) ==> Aggreg_PROD

PRINCIPE :

Traitement successif des 3 types de requêtes (avec en bonus les carac de l'installation)
- Formulation et test de la requêtes
- Définition des dates:
    - Déterminer la liste des dates maximale
    - A partir des logs, déterminer les dates pour lesquelles on a déjà les données
    - Etablir la nouvelle liste de dates pour lesquelles on doit requêter (nouvelles dates ou erreurs)
- Boucle de requêtes
    - Charger le DF sur disque
    - Envois requête
    - Append des données reçues dans le DF
- Exporter les résultat prk et csv)
    - Logs
    - Data

LOGS :
v2.2 Ajout de la gestion des dates déjà téléchargés

TODO : 
- le rappel des date dl ne fonctionne pas comme il faut
- la borne haute des dates à appelée n'est pas call (exclusif au lieu d'inclusif)
- appliquer la refactoriser en fonctions au autres url
    - <<Attention ! pr les données aggrégées il va falloir reprendre des choses
- convertir les types à la création des df
- merge les dates et time en un TS ?

- Gestion des limites d'API
     - "Rate limit information can be obtained by setting the HTTP header X-Rate-Limit to 1" https://pvoutput.org/help.html

# Librairies et MeP

In [1]:
# Librairie

#Divers
import requests #Call API
import time #Pr avoir dates avec today etc
import os #Pr création dossier etc
import pandas as pd

#Graphiques

In [2]:
# Mise en forme du notebook 

# Nb de ligne à afficher #Uniquement pr Jup Notbook ?
# pd.options.display.max_rows=100 

# Largeur des cellules #Uniquement pr Jup Notbook ?
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:80% !important; }</style>"))

# Mettre en output toutes les sortie d'une cellule
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all" #"last_expr"

# Avoir une forme de débugger
from IPython.core.debugger import Tracer # Plancer Tracer()() dans la fonction permet d'avoir un pas à pas, "n(ext) line and run this one, c(ontinue) running until next breakpoint, q(uit) the debugger"

# Paramètres globaux

In [3]:
# Paramètres génériques des requètes

notre_sys_id = '&sid=66192'
api_key = '&key=a29f3f9a012ea9da77c829911b595ae7c470362b'
target_sys_id = '&sid1=13412'

para_gene = notre_sys_id+api_key+target_sys_id

## Bornes dates

In [4]:
# Valeurs pr production

wait = 13 #"300 requests per hour in donation mode." soit un await minimun de 12s
date_recente = pd.to_datetime('today')
date_ancienne = pd.to_datetime('20121114', format="%Y%m%d", errors='coerce') # C'est la première date à partir de laquelle des données exploitables existent

# Définition des fonctions

## Test du call API

In [5]:
def TEST_CALL_API(url, date="20130101", colonnes=[]):
    ''' url, date(opt), colonnes(opt) --> df
    url (url sans la date cible), 
    date (format texte, par défaut 20130101),
    colonnes (liste de noms de colonnes)
    '''
    #Tracer()() 
    reponse_l = []
    
    for ligne in requests.get(url+'&d='+date).text.split(';'):
        reponse_l.append(ligne.split(','))
        
    df = pd.DataFrame.from_records(reponse_l)
    
    if len(colonnes) == df.shape[1]:
        df.columns = colonnes

    return df.head()

## Instanciation (potenciellement par chargement des données déjà DL)

In [6]:
def CHARGEMENT_DONNEES_DISQUE(path, file_name_core, logs_colonnes_l, data_colonnes_l):
    '''(path, file_name_core) --> (log_df, data_df)
    NB : si les fichiers ne sont pas trouvés, on instancie des df vierges, mais qui ne seront pas écris sur disque ici. Ce sera la fonction d'enregistrement, qui dans la même passe attribues des noms de colonnes
    '''
    # Dossier
    if os.path.exists(path):
        print("Dossier %s : trouvé" % path)
    else:
        os.makedirs(path)
        print("Dossier %s : non trouvé" % path)

    # Fichier de logs :
    if os.path.isfile(path + file_name_core +"_log.pkl") and os.access(path + file_name_core + "_log.pkl", os.R_OK):
        print("Fichier %s : trouvé et chargé " % (file_name_core +"_log.pkl"))
        log_df = pd.read_pickle(path + file_name_core + "_log.pkl")
    else:
        print("Fichier %s : non trouvé, un nouveau est instancié" % (file_name_core +"_log.pkl"))
        log_df = pd.DataFrame(columns= logs_colonnes_l)
        

    # Fichier de données :
    if os.path.isfile(path + file_name_core +"_data.pkl") and os.access(path + file_name_core + "_data.pkl", os.R_OK):
        print("Fichier %s : trouvé et chargé " % (file_name_core +"_data.pkl"))
        data_df = pd.read_pickle(path + file_name_core + "_data.pkl")
    else:
        print("Fichier %s : non trouvé, un nouveau est instancié" % (file_name_core +"_data.pkl"))
        data_df = pd.DataFrame(columns = data_colonnes_l)
    
    return (log_df, data_df)

## Création liste de dates à call

In [7]:
def FUNC_DATES_TO_CALL(log_df): #<<ATTENTION a adapter pr données de production aggrégées, dont la requête n'utilise pas la même forme de liste de date
    '''log --> dates_requetes_l
    '''
    
    # Instanciation de la liste de date (en str) à paser argument de la requête :
    dates_possibles_l = pd.date_range(start= date_ancienne, end= date_recente).strftime('%Y%m%d')
    dates_deja_obtenues_l = list(log_df.loc[log_df['Réponse'] == 200, 'Date_cible'])
    dates_requetes_l = [date for date in dates_possibles_l if date not in dates_deja_obtenues_l]
    #print("dates_possibles_l", type(dates_possibles_l), dates_possibles_l) 
    #print("dates_deja_obtenues_l", type(dates_deja_obtenues_l), dates_deja_obtenues_l)
    #print("dates_requetes_l", type(dates_requetes_l), dates_requetes_l)

    # Calcul durée traitement (en heures)
    print("Il y a déjà %d dates d'obtenues sur %d, soit encore %d dates a obtenir" % (len(dates_deja_obtenues_l), len(dates_possibles_l), len(dates_requetes_l)))
    print("Traiter ce batch prendra %.2f heures" % ((len(dates_requetes_l)*wait)/(60*60)))
    
    return dates_requetes_l

## Call API en batchs

In [8]:
def FUNC_CALL_API_JOUR(dates_requetes_l, wait, url, log_df, data_df):
    '''(dates_requetes_l, wait, url, log_df, data_df) --> (log_df, data_df) complétés
    '''

    for date in dates_requetes_l :
        #Limitateur débit
        time.sleep(wait) #"300 requests per hour in donation mode." soit un await minimun de 12s

        #Envois requête
        requete = requests.get(url + '&d=' + date)
        
        #Affichage live
        print("Dates en cours: de ", date, "\t",requete.status_code)

        #Log des calls
        log_df = log_df.append(
            dict(
                zip(
                    log_df.columns, 
                    [pd.to_datetime('today'), date, requete.status_code]
                )
            ), 
            ignore_index=True
        )

       #Controleur validité réponse
        if requete.status_code == 200:
            for ligne in requete.text.split(';'):
                #Append du df
                data_df = data_df.append(
                    dict(
                        zip(
                            data_df, 
                            (ligne.split(',') + [date]) # Petite trick due à la requête pr l'insolation, qui ne renvoit pas la date (==> on force la présence de la colonne et ici sa valeur)
                        )
                    ),
                    ignore_index=True
                )
            
        
    return (log_df, data_df)

## Enregistrement sur disque

In [9]:
def FUNC_ENREGISTREMENT_DISQUE(path, file_name_core, log_df, data_df):
    '''(path, file_name_core, log_df, data_df) --> Enregistrement de log_df et data_df en pkl et csv dans le dossier spécifié
    '''
    # Logs
    log_df.to_pickle(path + file_name_core + "_log.pkl")
    log_df.to_csv(path + file_name_core + "_log.csv", index= False)
    
    # Df
    data_df.to_pickle(path + file_name_core + "_data.pkl")
    data_df.to_csv(path + file_name_core + "_data.csv", index= False)
    
    print("Fichiers enregistrés")

## Check du df

In [10]:
def CHECK_DF(df):
    '''df --> Nb de lignes, nb de lignes en double, df.info() et df.head(15) 
    '''
    print("nb de ligne : ", len(df))
    print("nb de ligne en doublons : ", df.duplicated().sum(), "\n")
    
    df.info()
    return df.head(15)

## Set de valeur pour débugage

# Détail_PRODUCTION

Détail de la production d'un jour (getstatus).
Résolution temporelle de 10min.

In [11]:
# Paramètres pour "Détails Production"

# Divers
detail_PRODUCTION_path = "./DETAIL_PRODUCTION/" #Attention à ce fichu / en fin de path ;)
detail_PRODUCTION_file_name_core = "detail_PRODUCTION"
detail_PRODUCTION_logs_colonnes_l = ["TS_Call", "Date_cible", "Réponse"]
detail_PRODUCTION_data_colonnes_l  = ["Date", "Time", "Energy_Generation", "Energy_Efficiency", "Instantaneous_Power", "Average_Power", "Normalised_Output", "Energy_Consumption", "Power_Consumption", "Temperature", "Voltage"] #/ The parameter sid1 is able to retrieve generation data from any system. Consumption data is not returned. The requesting system must have donation mode enabled. 

# URL
detail = '&h=1' #The history parameter returns the entire status for a given date. 
nb_max = '&limit=288' #Besoin de fixer à la valeur max de 288
ordre = '&asc=1'
detail_PRODUCTION_url = 'https://pvoutput.org/service/r2/getstatus.jsp?'+detail+nb_max+ordre+para_gene

In [12]:
# Test
reponse = requests.get(detail_PRODUCTION_url+'&d='+"20121114")
reponse.status_code
reponse.text.split(';')

200

['20121114,08:50,0,0.000,26,NaN,NaN,NaN,NaN,5.8,NaN',
 '20121114,09:00,5,0.005,32,30,0.028,NaN,NaN,5.8,NaN',
 '20121114,09:10,11,0.010,46,36,0.033,NaN,NaN,5.8,NaN',
 '20121114,09:20,23,0.021,80,72,0.067,NaN,NaN,6.1,NaN',
 '20121114,09:30,38,0.035,96,90,0.083,NaN,NaN,6.1,NaN',
 '20121114,09:40,55,0.051,140,102,0.094,NaN,NaN,6.1,NaN',
 '20121114,09:50,84,0.078,237,174,0.161,NaN,NaN,7.2,NaN',
 '20121114,10:00,146,0.135,467,372,0.344,NaN,NaN,7.2,NaN',
 '20121114,10:10,226,0.209,495,480,0.444,NaN,NaN,7.2,NaN',
 '20121114,10:20,311,0.288,519,510,0.472,NaN,NaN,8.4,NaN',
 '20121114,10:30,400,0.370,545,534,0.494,NaN,NaN,8.4,NaN',
 '20121114,10:40,493,0.456,571,558,0.517,NaN,NaN,8.4,NaN',
 '20121114,10:50,589,0.545,589,576,0.533,NaN,NaN,9.8,NaN',
 '20121114,11:00,689,0.638,606,600,0.556,NaN,NaN,9.8,NaN',
 '20121114,11:10,792,0.733,622,618,0.572,NaN,NaN,9.8,NaN',
 '20121114,11:20,897,0.831,631,630,0.583,NaN,NaN,12.2,NaN',
 '20121114,11:30,1003,0.929,639,636,0.589,NaN,NaN,12.2,NaN',
 '20121114,11:

Traitement du batch

In [13]:
# Test du call à l'API pour cette url 
TEST_CALL_API(
    detail_PRODUCTION_url, 
    colonnes = detail_PRODUCTION_data_colonnes_l)

# Chargement / Instanciation des df :
(detail_PRODUCTION_log_df, detail_PRODUCTION_data_df) = CHARGEMENT_DONNEES_DISQUE(
    detail_PRODUCTION_path, 
    detail_PRODUCTION_file_name_core,
    detail_PRODUCTION_logs_colonnes_l, 
    detail_PRODUCTION_data_colonnes_l
)

# Instanciation de la liste des dates manquantes :
detail_PRODUCTION_dates_requetes_l = FUNC_DATES_TO_CALL(
    detail_PRODUCTION_log_df)

# Call de l'API pr les dates manquantes : 
(detail_PRODUCTION_log_df, detail_PRODUCTION_data_df) = FUNC_CALL_API_JOUR(
    detail_PRODUCTION_dates_requetes_l, 
    wait, 
    detail_PRODUCTION_url, 
    detail_PRODUCTION_log_df, 
    detail_PRODUCTION_data_df)

# Enregistrement sur disque :
FUNC_ENREGISTREMENT_DISQUE(
    detail_PRODUCTION_path, 
    detail_PRODUCTION_file_name_core, 
    detail_PRODUCTION_log_df, 
    detail_PRODUCTION_data_df)

# Check du dataset dispo :
CHECK_DF(detail_PRODUCTION_data_df)

,Date,Time,Energy_Generation,Energy_Efficiency,Instantaneous_Power,Average_Power,Normalised_Output,Energy_Consumption,Power_Consumption,Temperature,Voltage
0,20130101,10:00,0,0.000,25,NaN,NaN,NaN,NaN,8.6,NaN
1,20130101,10:10,4,0.004,25,24,0.022,NaN,NaN,8.9,NaN
2,20130101,10:20,10,0.009,52,36,0.033,NaN,NaN,8.9,NaN
3,20130101,10:30,19,0.018,70,54,0.050,NaN,NaN,8.9,NaN
4,20130101,10:40,27,0.025,34,48,0.044,NaN,NaN,9.5,NaN


Dossier ./DETAIL_PRODUCTION/ : trouvé
Fichier detail_PRODUCTION_log.pkl : trouvé et chargé 
Fichier detail_PRODUCTION_data.pkl : trouvé et chargé 
Il y a déjà 2307 dates d'obtenues sur 2438, soit encore 131 dates a obtenir
Traiter ce batch prendra 0.47 heures
Dates en cours: de  20121205 	 400
Dates en cours: de  20121206 	 400
Dates en cours: de  20121207 	 400
Dates en cours: de  20121208 	 400
Dates en cours: de  20121209 	 400
Dates en cours: de  20121210 	 400
Dates en cours: de  20121211 	 400
Dates en cours: de  20121212 	 400
Dates en cours: de  20121213 	 400
Dates en cours: de  20121214 	 400
Dates en cours: de  20130313 	 400
Dates en cours: de  20131008 	 400
Dates en cours: de  20131009 	 400
Dates en cours: de  20131010 	 400
Dates en cours: de  20131011 	 400
Dates en cours: de  20131012 	 400
Dates en cours: de  20131013 	 400
Dates en cours: de  20131014 	 400
Dates en cours: de  20141007 	 400
Dates en cours: de  20141008 	 400
Dates en cours: de  20141009 	 400
Dates

,Date,Time,Energy_Generation,Energy_Efficiency,Instantaneous_Power,Average_Power,Normalised_Output,Energy_Consumption,Power_Consumption,Temperature,Voltage
0,20121114,08:50,0,0.000,26,NaN,NaN,NaN,NaN,5.8,NaN
1,20121114,09:00,5,0.005,32,30,0.028,NaN,NaN,5.8,NaN
2,20121114,09:10,11,0.010,46,36,0.033,NaN,NaN,5.8,NaN
3,20121114,09:20,23,0.021,80,72,0.067,NaN,NaN,6.1,NaN
4,20121114,09:30,38,0.035,96,90,0.083,NaN,NaN,6.1,NaN
5,20121114,09:40,55,0.051,140,102,0.094,NaN,NaN,6.1,NaN
6,20121114,09:50,84,0.078,237,174,0.161,NaN,NaN,7.2,NaN
7,20121114,10:00,146,0.135,467,372,0.344,NaN,NaN,7.2,NaN
8,20121114,10:10,226,0.209,495,480,0.444,NaN,NaN,7.2,NaN
9,20121114,10:20,311,0.288,519,510,0.472,NaN,NaN,8.4,NaN


# Détail_INSOLATION

Détail de l'insolation d'un jour (getinsolation).
Résolution temporelle de 5min.

In [14]:
# Paramètres pour "Détails Insolation"

# Divers
detail_INSOLATION_path = "./DETAIL_INSOLATION/" #Attention à ce fichu / en fin de path ;)
detail_INSOLATION_file_name_core = "detail_INSOLATION"
detail_INSOLATION_logs_colonnes_l = ["TS_Call", "Date_cible", "Réponse"]
detail_INSOLATION_data_colonnes_l  = ["Time", "Power", "Energy", "Date"] #Il faut injecter manuellement la date, comme le call ne peut porter que sur une journée cible, c'est omis ds la réponse !

# URL 
#Pas de paramètres interessants
detail_INSOLATION_url = 'https://pvoutput.org/service/r2/getinsolation.jsp?'+para_gene

In [15]:
# Test du call à l'API pour cette url 
TEST_CALL_API(
    detail_INSOLATION_url, 
    colonnes = detail_INSOLATION_data_colonnes_l)

# Chargement / Instanciation des df :
(detail_INSOLATION_log_df, detail_INSOLATION_data_df) = CHARGEMENT_DONNEES_DISQUE(
    detail_INSOLATION_path, 
    detail_INSOLATION_file_name_core,
    detail_INSOLATION_logs_colonnes_l, 
    detail_INSOLATION_data_colonnes_l)

# Instanciation de la liste des dates manquantes :
detail_INSOLATION_dates_requetes_l = FUNC_DATES_TO_CALL(
    detail_INSOLATION_log_df)

# Call de l'API pr les dates manquantes : 
(detail_INSOLATION_log_df, detail_INSOLATION_data_df) = FUNC_CALL_API_JOUR(
    detail_INSOLATION_dates_requetes_l, 
    wait, 
    detail_INSOLATION_url, 
    detail_INSOLATION_log_df, 
    detail_INSOLATION_data_df)

# Enregistrement sur disque :
FUNC_ENREGISTREMENT_DISQUE(
    detail_INSOLATION_path, 
    detail_INSOLATION_file_name_core,
    detail_INSOLATION_log_df, 
    detail_INSOLATION_data_df)

# Check du dataset dispo :
CHECK_DF(
    detail_INSOLATION_data_df)

,0,1,2
0,08:40,0,0
1,08:45,5,0
2,08:50,18,2
3,08:55,35,5
4,09:00,54,9


Dossier ./DETAIL_INSOLATION/ : trouvé
Fichier detail_INSOLATION_log.pkl : trouvé et chargé 
Fichier detail_INSOLATION_data.pkl : trouvé et chargé 
Il y a déjà 2358 dates d'obtenues sur 2438, soit encore 80 dates a obtenir
Traiter ce batch prendra 0.29 heures
Dates en cours: de  20190430 	 200
Dates en cours: de  20190501 	 200
Dates en cours: de  20190502 	 200
Dates en cours: de  20190503 	 200
Dates en cours: de  20190504 	 200
Dates en cours: de  20190505 	 200
Dates en cours: de  20190506 	 200
Dates en cours: de  20190507 	 200
Dates en cours: de  20190508 	 200
Dates en cours: de  20190509 	 200
Dates en cours: de  20190510 	 200
Dates en cours: de  20190511 	 200
Dates en cours: de  20190512 	 200
Dates en cours: de  20190513 	 200
Dates en cours: de  20190514 	 200
Dates en cours: de  20190515 	 200
Dates en cours: de  20190516 	 200
Dates en cours: de  20190517 	 200
Dates en cours: de  20190518 	 200
Dates en cours: de  20190519 	 200
Dates en cours: de  20190520 	 200
Dates 

,Time,Power,Energy,Date
0,08:00,0,0,20121114
1,08:05,6,1,20121114
2,08:10,19,2,20121114
3,08:15,35,5,20121114
4,08:20,54,10,20121114
5,08:25,73,16,20121114
6,08:30,93,23,20121114
7,08:35,113,33,20121114
8,08:40,133,44,20121114
9,08:45,153,57,20121114


# Aggrég_PRODUCTION

Set de valeurs aggrégées de la production sur une journée (getoutput).
Résolution temporelle de 1j.

Todo :
- Charger valeurs fichiers (pas réellement besoin, mais pr faire propre)
- Mettre aux propres les arguments des fonctions (celles de débug actuellement présentes)

In [16]:
# Paramètres pour "Détails Insolation"

# Divers
aggreg_PRODUCTION_path = "./AGGREGATION_PRODUCTION/" #Attention à ce fichu / en fin de path ;)
aggreg_PRODUCTION_file_name_core = "aggreg_PRODUCTION"
aggreg_PRODUCTION_logs_colonnes_l = ["TS_Call", "Date_Début_Période", "Date_Fin_Période", "Nb_j", "Réponse"]
aggreg_PRODUCTION_data_colonnes_l  = ["Date", "Energy_Generated", "Efficiency", "Energy_Exported", "Energy_Used", "Peak_Power", "Peak_Time", "Condition", "Min_Temperature", "Max_Temperature", "Peak_Energy_Import", "Off-Peak_Energy_Import", "Shoulder_Energy_Import", "High-Shoulder_Energy_Import", "Insolation"]

# URL 
insolation = '&insolation=1'
nb_max = '&limit=150' #C'est une limite de l'API pr ce call

aggreg_PRODUCTION_url = 'https://pvoutput.org/service/r2/getoutput.jsp?'+insolation+nb_max+para_gene

In [17]:
dates_l = list(pd.date_range(start= date_ancienne, end= pd.to_datetime('today'), freq='1D').strftime('%Y%m%d'))
print("nb jours sur l'interval entier", len(dates_l))
print("nb de batch (de 150j) à envoyer", len(dates_l) / 150)

#ID_interval = 0
#
#for i in range(0, len(dates_l), 151):
#    print("Soit l'intervale : ", ID_interval)
#    ID_interval += 1
#    
#    if (i+150) < len(dates_l):
#        print("début : " + str(dates_l[i]) + " fin : " + str(dates_l[i+150]) + " soit 150j")
#    else: 
#        print("début : " + str(dates_l[i]) + " fin : " + str(dates_l[-1]) + " soit " + str(len(dates_l)-1-i) + "j")

nb jours sur l'interval entier 2438
nb de batch (de 150j) à envoyer 16.253333333333334


In [18]:
log_df = pd.DataFrame(columns= aggreg_PRODUCTION_logs_colonnes_l)
data_df = pd.DataFrame(columns= aggreg_PRODUCTION_data_colonnes_l)

# Call API

for i in range(0, len(dates_l), 151):
    #Limitateur débit
    time.sleep(wait) #"300 requests per hour in donation mode." soit un await minimun de 12s
    
    # Routage pour dernière période 
    if (i+150) <= len(dates_l):
        fin = dates_l[i+150]
        nb_j = 150
        
    if (i+150) > len(dates_l):
        fin = dates_l[-1]
        nb_j = len(dates_l)-i
    #<<Ajouter un catch d'erreur?

    #Envois requête
    requete = requests.get(url + '&df=' + dates_l[i] + '&dt=' + fin) #Il semble que j'avais "d" au lieux de "df", erreur ou contournement ?

    #Affichage live
    print("De {} à {} soit {} j, statut {} ".format(dates_l[i], fin, nb_j, requete.status_code))
    
    #Log des calls
    log_df = log_df.append(
        dict(
            zip(
                log_df.columns, #["TS_Call", "Date_Début_Période", "Date_Fin_Période", "Nb_j", "Réponse"]
                [pd.to_datetime('today'), dates_l[i], fin, nb_j, requete.status_code]
            )
        ), 
        ignore_index=True
    )


    for ligne in requete.text.split(';'):
        #Append du df
        data_df = data_df.append(
            dict(
                zip(
                    data_df, 
                    ligne.split(',')
                )
            ),
            ignore_index=True
        )

data_df.sort_values(by='Date', inplace= True)  

NameError: name 'url' is not defined

In [ ]:
# Enregistrement sur disque 

FUNC_ENREGISTREMENT_DISQUE(path, file_name_core, log_df, data_df)

In [ ]:
# Check des données

CHECK_DF(data_df)

# Info installation photovoiltaique

In [ ]:
# url info instalation

detail = '&ext=1' 
sec_ar = '&array2=1'

info_instalation_url = 'https://pvoutput.org/service/r2/getsystem.jsp?'+detail+sec_ar+para_gene

In [ ]:
#Test requete instalation

requests.get(info_instalation_url).text